In [30]:
import pickle
import os

cache_dir = os.path.join("./cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        print("CACHE NOT FOUND")
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [31]:
train_X, test_X, train_y, test_y = preprocess_data()
type(train_X), type(train_y)
train_X.extend(test_X)
train_y.extend(test_y)
len(train_X), len(train_y)

Read preprocessed data from cache file: preprocessed_data.pkl


(50000, 50000)

In [7]:
import random
type(train_X), type(train_y)
temp = list(zip(train_X, train_y))
random.shuffle(temp)
train_X, train_y = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
train_X, train_y = list(train_X), list(train_y)

In [8]:

type(train_X), type(train_y)
print(len(train_X))
train_list = train_X[:30000]
train_labels_list = train_y[:30000]
test_list = train_X[30000:]
test_labels_list = train_y[30000:]
print(len(train_list), len(train_labels_list), len(test_list), len(test_labels_list))
print(train_list[0], test_list[0], train_labels_list[0], test_labels_list[0])

50000
30000 30000 20000 20000
['basic', 'like', 'verhoeven', 'film', 'film', 'enjoy', 'brilliant', 'pscychosexu', 'stori', 'seen', 'basic', 'instinct', 'realli', 'wonder', 'thriller', 'enjoy', 'much', 'obvious', 'watch', 'anoth', 'verhoeven', 'movi', 'well', 'previou', 'direct', 'block', 'buster', 'hit', 'basic', 'instinct', 'much', 'curiou', 'watch', 'movi', 'yeah', 'movi', 'fulfil', 'hope', 'expect', 'movi', 'fourth', 'man', 'brilliant', 'pscychosexu', 'drama', 'lit', 'bit', 'complex', 'audienc', 'stori', 'movi', 'gay', 'writer', 'name', 'reve', 'krabb', 'alcohol', 'person', 'live', 'moral', 'valu', 'see', 'mani', 'vision', 'may', 'warn', 'futur', 'accid', 'end', 'lectur', 'introduc', 'seduct', 'woman', 'name', 'christin', 'mysteri', 'past', 'want', 'reveal', 'reve', 'sex', 'hous', 'boy', 'next', 'morn', 'watch', 'sexi', 'macho', 'boyfriend', 'pictur', 'tabl', 'person', 'met', 'station', 'curiou', 'meet', 'tell', 'christin', 'invit', 'hous', 'want', 'reveal', 'entir', 'stori', 'verho

In [32]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        for word in sentence:
            if word in word_count.keys():
                word_count[word] = word_count[word] + 1
            else:
                word_count[word] = 1
   
    sorted_words = {k: v for k, v in sorted(word_count.items(), key=lambda item: item[1], reverse=True)}
    sorted_words = [word for word in sorted_words.keys()]
    word_dict = {} # This is what we are building, a dictionary that translates words into integers

    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [ ]:
word_dict = build_dict(train_list)

In [35]:
data_dir = './data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [13]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_list)
test_X, test_X_len = convert_and_pad_data(word_dict, test_list)


In [14]:
print(train_X.shape, len(train_X_len))
print(test_X.shape, len(test_X_len))
print(train_y[0])

(30000, 500) 30000
(20000, 500) 20000
1


In [15]:
import pandas as pd
pd.concat([pd.DataFrame(train_labels_list), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1).to_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=False, index=False)

In [16]:
pd.concat([pd.DataFrame(test_labels_list), pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1).to_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=False, index=False)

# If files ae present start from here

In [120]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)


In [121]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((30000, 502), (10000, 502), (10000, 502))

In [122]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000, 1])


In [135]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self,vocab_size,embed_size,filter_size,kernel_size,dropout,seq_len):
        super(CNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size,embed_size)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],embed_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],embed_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],embed_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dense = torch.nn.Linear(filter_size*3,1)
        self.dropout = torch.nn.Dropout(dropout)
    
    def forward(self, x, target):
        embed_input = self.embedding(x)
        embed_input.unsqueeze_(1)
        x1 = torch.tanh(self.dropout(self.conv1(embed_input))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(embed_input))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(embed_input))).squeeze(3)
        f1 = self.mp1(x1).squeeze(2)
        f2 = self.mp2(x2).squeeze(2)
        f3 = self.mp3(x3).squeeze(2)
        hidden = torch.cat([f1,f2,f3],dim=1)
        logits = self.dense(hidden)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [ ]:
# write to file.
filename = "CNN_imdb.csv"
def write_to_csv(epochs, train_loss, val_loss, val_acc, end):
    epoch = [i for i in range(10)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, val_loss, val_acc, end)), columns =['Epoch', 'train_loss', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(file_name)
    df_csv['Test_Accuracy']  = accuracy
    df_metrics.to_csv(filename)

In [136]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
embed_size = 300
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 5000
embed_dims = 32
kernel_size = [3,4,5]
model = CNN(vocab_size,embed_dims,filter_size,kernel_size,dropout,seq_len).to(device)

In [138]:
import time

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
counter = 0
CNN_acc = []
CNN_valacc = []
train_loss_epoch = []
val_loss_epoch = []
val_acc_epoch = []
time_epoch = []
model.train()
for e in range(epochs):
    start_time = time.time()
    for inputs, labels in train_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        model.zero_grad()
        logits, accuracy = model(inputs,labels)
        loss = criterion(logits,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        if counter%10==0:
            print("Epoch: {}/{}".format(e,epochs),
                         "\tIteration: {}".format(counter),
                         "\t\tTrain Loss: {:.3f}".format(loss.item()),
                         "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
            CNN_acc.append(accuracy.item()*100/batch_size)
        counter += 1
            
    train_loss_epoch.append(np.mean(train_loss))     
    print("Epoch: {}/{}".format(e,epochs), "\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))

    with torch.no_grad():
        model.eval()
        val_acc = []
        val_loss = []
        for inputs, labels in val_dl:
            inputs_val, labels_val = inputs.cuda(), labels.cuda()
            logits_val,accuracy_val = model(inputs_val,labels_val)
            loss_val = criterion(logits_val,labels_val.float())
            val_acc.append(accuracy_val.item()*100/batch_size)
            val_loss.append(loss_val.item())
        val_loss_epoch.append(np.mean(val_loss))
        val_acc_epoch.append(np.mean(val_acc))
        print("\t\tVal Loss: {:.3f}".format(np.mean(val_loss)), "\t\tVal Acc: {:.3f}".format(np.mean(val_acc)))
        CNN_valacc.append(np.mean(val_acc))
        model.train()
        
    end_time = time.time()-start_time
    print("Time to train epoch: {0} s".format(end_time))
    time_epoch.append(end_time)
    
write_to_csv(epochs, train_loss_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
    
with torch.no_grad():
    model.eval()
    test_acc = []
    test_loss = []
    for inputs, labels in test_dl:
        input_test, labels_test = inputs.cuda(), labels.cuda()
        logits_test,accuracy_test = model(input_test,labels_test)
        loss_test = criterion(logits_test,labels_test.float())
        test_acc.append(accuracy_test.item()*100/batch_size)
        test_loss.append(loss_test.item())
    print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))
    append_to_csv(epochs, np.mean(test_acc))

Epoch: 0/10 	Iteration: 0 		Train Loss: 0.698 	Train Accuracy: 50.00
		Val Loss: 0.694 		Val Acc: 49.850
Epoch: 0/10 	Iteration: 10 		Train Loss: 0.700 	Train Accuracy: 44.00
Epoch: 0/10 	Iteration: 20 		Train Loss: 0.695 	Train Accuracy: 48.00
Epoch: 0/10 	Iteration: 30 		Train Loss: 0.670 	Train Accuracy: 60.00
Epoch: 0/10 	Iteration: 40 		Train Loss: 0.692 	Train Accuracy: 58.00
Epoch: 0/10 	Iteration: 50 		Train Loss: 0.702 	Train Accuracy: 48.00
Epoch: 0/10 	Iteration: 60 		Train Loss: 0.700 	Train Accuracy: 40.00
Epoch: 0/10 	Iteration: 70 		Train Loss: 0.685 	Train Accuracy: 54.00
Epoch: 0/10 	Iteration: 80 		Train Loss: 0.684 	Train Accuracy: 64.00
Epoch: 0/10 	Iteration: 90 		Train Loss: 0.705 	Train Accuracy: 48.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.697 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 110 		Train Loss: 0.703 	Train Accuracy: 48.00
Epoch: 0/10 	Iteration: 120 		Train Loss: 0.697 	Train Accuracy: 44.00
Epoch: 0/10 	Iteration: 130 		Train Loss: 0.698 	Tra

Epoch: 1/10 	Iteration: 1140 		Train Loss: 0.295 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 1150 		Train Loss: 0.379 	Train Accuracy: 80.00
Epoch: 1/10 	Iteration: 1160 		Train Loss: 0.364 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 1170 		Train Loss: 0.226 	Train Accuracy: 94.00
Epoch: 1/10 	Iteration: 1180 		Train Loss: 0.357 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 1190 		Train Loss: 0.363 	Train Accuracy: 82.00
		Val Loss: 0.351 		Val Acc: 85.010
Time to train epoch: 131.97090697288513 s
Epoch: 2/10 	Iteration: 1200 		Train Loss: 0.306 	Train Accuracy: 90.00
Epoch: 2/10 	Iteration: 1210 		Train Loss: 0.280 	Train Accuracy: 88.00
Epoch: 2/10 	Iteration: 1220 		Train Loss: 0.410 	Train Accuracy: 76.00
Epoch: 2/10 	Iteration: 1230 		Train Loss: 0.217 	Train Accuracy: 92.00
Epoch: 2/10 	Iteration: 1240 		Train Loss: 0.244 	Train Accuracy: 86.00
Epoch: 2/10 	Iteration: 1250 		Train Loss: 0.390 	Train Accuracy: 78.00
Epoch: 2/10 	Iteration: 1260 		Train Loss: 0.263 	Train Ac

Epoch: 3/10 	Iteration: 2260 		Train Loss: 0.208 	Train Accuracy: 94.00
Epoch: 3/10 	Iteration: 2270 		Train Loss: 0.406 	Train Accuracy: 84.00
Epoch: 3/10 	Iteration: 2280 		Train Loss: 0.145 	Train Accuracy: 96.00
Epoch: 3/10 	Iteration: 2290 		Train Loss: 0.467 	Train Accuracy: 82.00
Epoch: 3/10 	Iteration: 2300 		Train Loss: 0.185 	Train Accuracy: 96.00
Epoch: 3/10 	Iteration: 2310 		Train Loss: 0.405 	Train Accuracy: 82.00
Epoch: 3/10 	Iteration: 2320 		Train Loss: 0.224 	Train Accuracy: 90.00
Epoch: 3/10 	Iteration: 2330 		Train Loss: 0.297 	Train Accuracy: 86.00
Epoch: 3/10 	Iteration: 2340 		Train Loss: 0.226 	Train Accuracy: 92.00
Epoch: 3/10 	Iteration: 2350 		Train Loss: 0.250 	Train Accuracy: 86.00
Epoch: 3/10 	Iteration: 2360 		Train Loss: 0.290 	Train Accuracy: 90.00
Epoch: 3/10 	Iteration: 2370 		Train Loss: 0.172 	Train Accuracy: 96.00
Epoch: 3/10 	Iteration: 2380 		Train Loss: 0.251 	Train Accuracy: 90.00
Epoch: 3/10 	Iteration: 2390 		Train Loss: 0.280 	Train Accuracy

Epoch: 5/10 	Iteration: 3380 		Train Loss: 0.211 	Train Accuracy: 94.00
Epoch: 5/10 	Iteration: 3390 		Train Loss: 0.279 	Train Accuracy: 88.00
Epoch: 5/10 	Iteration: 3400 		Train Loss: 0.255 	Train Accuracy: 92.00
Epoch: 5/10 	Iteration: 3410 		Train Loss: 0.212 	Train Accuracy: 92.00
Epoch: 5/10 	Iteration: 3420 		Train Loss: 0.215 	Train Accuracy: 88.00
Epoch: 5/10 	Iteration: 3430 		Train Loss: 0.174 	Train Accuracy: 94.00
Epoch: 5/10 	Iteration: 3440 		Train Loss: 0.164 	Train Accuracy: 96.00
Epoch: 5/10 	Iteration: 3450 		Train Loss: 0.352 	Train Accuracy: 84.00
Epoch: 5/10 	Iteration: 3460 		Train Loss: 0.151 	Train Accuracy: 98.00
Epoch: 5/10 	Iteration: 3470 		Train Loss: 0.304 	Train Accuracy: 82.00
Epoch: 5/10 	Iteration: 3480 		Train Loss: 0.117 	Train Accuracy: 98.00
Epoch: 5/10 	Iteration: 3490 		Train Loss: 0.358 	Train Accuracy: 84.00
Epoch: 5/10 	Iteration: 3500 		Train Loss: 0.164 	Train Accuracy: 96.00
Epoch: 5/10 	Iteration: 3510 		Train Loss: 0.366 	Train Accuracy

Epoch: 7/10 	Iteration: 4500 		Train Loss: 0.152 	Train Accuracy: 94.00
Epoch: 7/10 	Iteration: 4510 		Train Loss: 0.060 	Train Accuracy: 100.00
Epoch: 7/10 	Iteration: 4520 		Train Loss: 0.097 	Train Accuracy: 98.00
Epoch: 7/10 	Iteration: 4530 		Train Loss: 0.221 	Train Accuracy: 92.00
Epoch: 7/10 	Iteration: 4540 		Train Loss: 0.098 	Train Accuracy: 98.00
Epoch: 7/10 	Iteration: 4550 		Train Loss: 0.123 	Train Accuracy: 96.00
Epoch: 7/10 	Iteration: 4560 		Train Loss: 0.190 	Train Accuracy: 90.00
Epoch: 7/10 	Iteration: 4570 		Train Loss: 0.179 	Train Accuracy: 92.00
Epoch: 7/10 	Iteration: 4580 		Train Loss: 0.220 	Train Accuracy: 94.00
Epoch: 7/10 	Iteration: 4590 		Train Loss: 0.218 	Train Accuracy: 94.00
Epoch: 7/10 	Iteration: 4600 		Train Loss: 0.143 	Train Accuracy: 94.00
Epoch: 7/10 	Iteration: 4610 		Train Loss: 0.214 	Train Accuracy: 92.00
Epoch: 7/10 	Iteration: 4620 		Train Loss: 0.211 	Train Accuracy: 88.00
Epoch: 7/10 	Iteration: 4630 		Train Loss: 0.145 	Train Accurac

Epoch: 9/10 	Iteration: 5620 		Train Loss: 0.185 	Train Accuracy: 94.00
Epoch: 9/10 	Iteration: 5630 		Train Loss: 0.104 	Train Accuracy: 96.00
Epoch: 9/10 	Iteration: 5640 		Train Loss: 0.156 	Train Accuracy: 90.00
Epoch: 9/10 	Iteration: 5650 		Train Loss: 0.199 	Train Accuracy: 94.00
Epoch: 9/10 	Iteration: 5660 		Train Loss: 0.303 	Train Accuracy: 90.00
Epoch: 9/10 	Iteration: 5670 		Train Loss: 0.157 	Train Accuracy: 94.00
Epoch: 9/10 	Iteration: 5680 		Train Loss: 0.159 	Train Accuracy: 98.00
Epoch: 9/10 	Iteration: 5690 		Train Loss: 0.175 	Train Accuracy: 96.00
Epoch: 9/10 	Iteration: 5700 		Train Loss: 0.163 	Train Accuracy: 94.00
Epoch: 9/10 	Iteration: 5710 		Train Loss: 0.059 	Train Accuracy: 100.00
Epoch: 9/10 	Iteration: 5720 		Train Loss: 0.094 	Train Accuracy: 98.00
Epoch: 9/10 	Iteration: 5730 		Train Loss: 0.235 	Train Accuracy: 88.00
Epoch: 9/10 	Iteration: 5740 		Train Loss: 0.124 	Train Accuracy: 96.00
Epoch: 9/10 	Iteration: 5750 		Train Loss: 0.134 	Train Accurac